# Bond Option Comparison with DerivaGem Function 17

Value an option on a coupon paying bond using the Hull-White in response to a question.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import time

In [2]:
from financepy.utils import *
from financepy.market.curves import *
from financepy.models.hw_tree import HWTree
from financepy.products.bonds import *

####################################################################
# FINANCEPY BETA Version 0.260 - This build:  22 Nov 2022 at 13:08 #
#      This software is distributed FREE & WITHOUT ANY WARRANTY    #
#  Report bugs as issues at https://github.com/domokane/FinancePy  #
####################################################################



## Set up the Discount Curve

In [3]:
settlement_date = Date(1, 12, 2019)

Set up discount curve

In [4]:
rate = 0.05
dcType = DayCountTypes.THIRTY_360_BOND
fixedFreq = FrequencyTypes.SEMI_ANNUAL
discount_curve = DiscountCurveFlat(settlement_date, rate, fixedFreq, dcType)

In [5]:
df_times = discount_curve._times
df_values = discount_curve._dfs

In [6]:
df_values

array([1.        , 0.9877296 , 0.97560976, 0.96363863, 0.9518144 ,
       0.94013525, 0.92859941, 0.91720512, 0.90595064, 0.89483426,
       0.88385429, 0.87300904, 0.86229687, 0.85171614, 0.84126524,
       0.83094257, 0.82074657, 0.81067568, 0.80072836, 0.7909031 ,
       0.7811984 , 0.77161278, 0.76214478, 0.75279296, 0.74355589,
       0.73443215, 0.72542038, 0.71651918, 0.7077272 , 0.6990431 ,
       0.69046556, 0.68199327, 0.67362493, 0.66535928, 0.65719506,
       0.64913101, 0.64116591, 0.63329855, 0.62552772, 0.61785224,
       0.61027094])

## Set up the Bond Option

First create the bond

In [7]:
issue_date = Date(1,12,2018)
maturity_date = issue_date.add_tenor("10Y")
coupon = 0.05
frequencyType = FrequencyTypes.SEMI_ANNUAL
accrual_type = DayCountTypes.THIRTY_360_BOND
bond = Bond(issue_date, maturity_date, coupon, frequencyType, accrual_type)

Let's first price the bond on the libor curve

In [8]:
cp = bond.clean_price_from_discount_curve(settlement_date, discount_curve)
dp = bond.dirty_price_from_discount_curve(settlement_date, discount_curve)
print("Fixed Income Clean Price: %9.3f"% cp)
print("Fixed Income Dirty Price: %9.3f"% dp)

Fixed Income Clean Price:   100.000
Fixed Income Full  Price:   102.500


In [9]:
settlement_date

01-DEC-2019

In [10]:
print(bond.coupon_dates(settlement_date))

 01-DEC-2019         2.50 
 01-JUN-2020         2.50 
 01-DEC-2020         2.50 
 01-JUN-2021         2.50 
 01-DEC-2021         2.50 
 01-JUN-2022         2.50 
 01-DEC-2022         2.50 
 01-JUN-2023         2.50 
 01-DEC-2023         2.50 
 01-JUN-2024         2.50 
 01-DEC-2024         2.50 
 01-JUN-2025         2.50 
 01-DEC-2025         2.50 
 01-JUN-2026         2.50 
 01-DEC-2026         2.50 
 01-JUN-2027         2.50 
 01-DEC-2027         2.50 
 01-JUN-2028         2.50 
 01-DEC-2028       102.50 



Then define and create the option

In [11]:
expiry_date = settlement_date.add_tenor("18m")
strike_price = 100.0
face_amount = 100.0

In [12]:
europeanCallBondOption = BondOption(bond, expiry_date, strike_price, face_amount, OptionTypes.EUROPEAN_CALL)
americanCallBondOption = BondOption(bond, expiry_date, strike_price, face_amount, OptionTypes.AMERICAN_CALL)

In [13]:
europeanPutBondOption = BondOption(bond, expiry_date, strike_price, face_amount, OptionTypes.EUROPEAN_PUT)
americanPutBondOption = BondOption(bond, expiry_date, strike_price, face_amount, OptionTypes.AMERICAN_PUT)

Consider the forward bond price.

In [14]:
cp = bond.clean_price_from_discount_curve(expiry_date, discount_curve)
dp = bond.dirty_price_from_discount_curve(expiry_date, discount_curve)
print("Fixed Income Clean Price: %9.3f"% cp)
print("Fixed Income Dirty Price: %9.3f"% dp)

Fixed Income Clean Price:   100.000
Fixed Income Full  Price:   102.500


Set the model parameters, start with zero vol.

In [15]:
num_steps = 500

In [16]:
sigma = 0.0125
a = 0.1
modelHW = HWTree(sigma, a, num_steps)

In [17]:
ec = europeanCallBondOption.value(settlement_date, discount_curve, modelHW)
ac = americanCallBondOption.value(settlement_date, discount_curve, modelHW)
print("European Call Value: %9.5f" % ec)
print("American Call Value: %9.5f" % ac)

European Call Value:   2.39763
American Call Value:   2.66598


In [18]:
ec = europeanPutBondOption.value(settlement_date, discount_curve, modelHW)
ac = americanPutBondOption.value(settlement_date, discount_curve, modelHW)
print("European Put Value: %9.5f" % ec)
print("American Put Value: %9.5f" % ac)

European Put Value:   2.42253
American Put Value:   2.68292


# COMPARISON

In [19]:
if 1==1:
    
    settlementDate = Date(1, 12, 2019)
    issueDate = Date(1, 12, 2018)
    expiryDate = settlementDate.add_tenor("18m")
    maturityDate = settlementDate.add_tenor("10Y")
    coupon = 0.05
    freqType = FrequencyTypes.SEMI_ANNUAL
    accrualType = DayCountTypes.THIRTY_360_BOND
    bond = Bond(issueDate, maturityDate, coupon, freqType, accrualType)

    couponTimes = []
    couponFlows = []
    cpn = bond._coupon/bond._frequency

    numFlows = len(bond._coupon_dates)
    for i in range(0, numFlows):

        pcd = bond._coupon_dates[i-1]
        ncd = bond._coupon_dates[i]

        if ncd > settlementDate:
            
            if len(couponTimes) == 0:
                flowTime = (pcd - settlementDate) / gDaysInYear
                couponTimes.append(flowTime)
                couponFlows.append(cpn)
                
            flowTime = (ncd - settlementDate) / gDaysInYear
            couponTimes.append(flowTime)
            couponFlows.append(cpn)

    couponTimes = np.array(couponTimes)
    couponFlows = np.array(couponFlows)

    strikePrice = 100.0
    face = 100.0

    model = HWTree(sigma, a, None)

    #  Test convergence
    texp = (expiryDate - settlementDate)/gDaysInYear
    tmat = (maturityDate - settlementDate)/gDaysInYear

    # Jamshidian approach
    vJam = model.european_bond_option_jamshidian(texp, strikePrice, face,
                                              couponTimes, couponFlows,
                                              df_times, df_values)

    model._num_time_steps = 200
    model.build_tree(tmat, df_times, df_values)
    exerciseType = FinExerciseTypes.EUROPEAN
    vHW = model.bond_option(texp, strikePrice, face,
                            couponTimes, couponFlows, exerciseType)


In [20]:
vJam

{'call': 2.557167911283727, 'put': 2.582591576448931}

In [21]:
vHW

{'call': 2.5651092396164996, 'put': 2.588240081574218}

Copyright (c) Dominic O'Kane 2021